In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC,LinearSVC
import numpy as np
import seaborn as sns
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split

## Support Vector Machines (lineal y polinomico)

In [2]:
#No recuerdo como utilizar el df directamente desde visual studio asi que lo baje a un csv y lo subi de vuelta
df = pd.read_csv("train_data_final.csv")



df.shape

df.columns

Index(['Unnamed: 0', 'customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1',
       'S_3', 'D_41',
       ...
       'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144',
       'D_145', 'target'],
      dtype='object', length=192)

In [3]:
categorical_variables = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 

df.drop(categorical_variables,axis=1,inplace=True)

payment_variables = []
spending_variables = []
delinquency_variables = []
balance_variables = []
risk_variables = []
for column in df.columns:
    if 'P' in column:
        payment_variables.append(column)
    if 'S' in column:
        spending_variables.append(column)
    if 'D' in column and 'ID' not in column:
        delinquency_variables.append(column)
    if 'B' in column:
        balance_variables.append(column)
    if 'R' in column:
        risk_variables.append(column)

In [4]:
df.drop('customer_ID',axis=1,inplace=True)

In [5]:
df = df.drop('S_2',axis=1)\
.drop('D_42',axis=1) \
.drop('D_49',axis=1) \
.drop('D_53',axis=1) \
.drop('D_73',axis=1) \
.drop('D_76',axis=1) \
.drop('R_9',axis=1)  \
.drop('D_82',axis=1) \
.drop('B_29',axis=1) \
.drop('D_87',axis=1) \
.drop('D_88',axis=1) \
.drop('D_106',axis=1)\
.drop('R_26',axis=1) \
.drop('D_108',axis=1)\
.drop('D_110',axis=1)\
.drop('D_111',axis=1)\
.drop('B_39',axis=1) \
.drop('B_42',axis=1) \
.drop('D_132',axis=1)\
.drop('D_134',axis=1)\
.drop('D_135',axis=1)\
.drop('D_136',axis=1)\
.drop('D_137',axis=1)\
.drop('D_138',axis=1)\
.drop('D_142',axis=1)\


delinquency_variables_redux = delinquency_variables.copy()



delinquency_variables_redux.remove('D_42') 
delinquency_variables_redux.remove('D_49') 
delinquency_variables_redux.remove('D_53') 
delinquency_variables_redux.remove('D_73') 
delinquency_variables_redux.remove('D_76') 
delinquency_variables_redux.remove('D_82') 
delinquency_variables_redux.remove('D_87') 
delinquency_variables_redux.remove('D_88') 
delinquency_variables_redux.remove('D_106')
delinquency_variables_redux.remove('D_108')
delinquency_variables_redux.remove('D_110')
delinquency_variables_redux.remove('D_111')
delinquency_variables_redux.remove('D_132')
delinquency_variables_redux.remove('D_134')
delinquency_variables_redux.remove('D_135')
delinquency_variables_redux.remove('D_136')
delinquency_variables_redux.remove('D_137')
delinquency_variables_redux.remove('D_138')
delinquency_variables_redux.remove('D_142')


df.loc[:,delinquency_variables_redux].fillna(0,inplace=True)

for col in df.columns:
    #print(col)
    df[col].fillna(df[col].mean(),inplace=True)

#### Armado del train_test

In [6]:
train_variables = []

for v in payment_variables:
    if v in df.columns:
        train_variables.append(v)
for v in balance_variables:
    if v in df.columns:        
        train_variables.append(v)
for v in risk_variables:
    if v in df.columns:        
        train_variables.append(v)     
for v in spending_variables:
    if v in df.columns:        
        train_variables.append(v)       
for v in delinquency_variables_redux:
    if v in df.columns:        
        train_variables.append(v)        
        
        

#X = df.iloc[0:20000,:][train_variables]
X = df[train_variables]
        
#X = df.drop(['target'],axis=1)
#X = X.drop(delinquency_variables_redux,axis=1)
#X = df[payment_variables]
#y = df.iloc[0:20000,:].target
y = df.target
features = X.columns





X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X_train_norm=scaler.fit_transform(X_train)
X_test_norm=scaler.fit_transform(X_test)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276572 entries, 0 to 276571
Columns: 153 entries, P_2 to D_145
dtypes: float64(152), int64(1)
memory usage: 322.8 MB


### Hiperparametros lineales y polinomicos para el svm

In [7]:
#Seteo hyperparametros lineales y polinomicas
param_dist_polinomic = {"kernel" : ['poly'] ,
            "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9],
            "degree": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4],
            "gamma": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4],
            "coef0": [1e-4,1e-3,1e-2,1e-1,1e+0,1e+1,1e+2,1e+3,1e+4]
              }

#param_dist_linear = {"kernel":['linear'],
#             "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9] }

param_dist_linear = {
             "C" : [1e+0,1e+1,1e+2,1e+3,1e+4,1e+5,1e+6,1e+7,1e+8,1e+9] }

In [8]:
random_search_polynomic_svc = RandomizedSearchCV(SVC(random_state=0),
                                       param_distributions = param_dist_polinomic,
                                       n_iter = 2,
                                       scoring = 'roc_auc',
                                       cv = 5,
                                       n_jobs = 2)



#random_search_linear_svc = RandomizedSearchCV(SVC(random_state=0,tol=1e-3,max_iter=20000),
#                                       param_distributions = param_dist_linear,
#                                       n_iter = 2,
#                                       scoring = 'roc_auc',
#                                       cv = 5,
#                                       n_jobs = 2)

parameters = {'kernel':('linear','rbf'),'C':[1,100]}

svc_gridsearch = SVC(random_state=0,tol=1e-3,max_iter=20000)

grid_search_svc = GridSearchCV(svc_gridsearch,parameters)

In [9]:
grid_search_svc.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__break_ties', 'estimator__cache_size', 'estimator__class_weight', 'estimator__coef0', 'estimator__decision_function_shape', 'estimator__degree', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__probability', 'estimator__random_state', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

## Entrenamiento de los modelos (sin probar porque no me los termina)

In [10]:
#%time

#grid_search_svc.fit(X_train_norm, y_train) #descomentar


#random_search_polynomic_svc.fit(X_train,y_train)

#### Conseguimos los nuevos hiperparametros

In [11]:
#params_opt_linear_svm = grid_search_svc.best_params_
#params_opt_linear_svm

#obtuve {'C': 1, 'kernel': 'rbf'} y lograba converger, con kernel= linear no.

#params_opt_polynomial_svm = random_search_polynomic_svc.best_params_

In [15]:
#linear_svc = SVC().set_params(**params_opt_linear_svm)
#linear_svc = SVC(C=1,tol=1e-3,max_iter=10000)
#polynomial_svc = SVC().set_params(**params_opt_polynomial_svm)

svc = SVC(C=1,tol=1e-2,kernel= 'rbf',max_iter=10000,random_state=0)


## Entrenamos de vuelta con svm y los nuevos parametros

In [ ]:
svc.fit(X_train_norm,y_train)

#polynomial_svc.fit(X_train,y_train)

#### Muestra de los resultados

In [14]:
#AGREGAR METODOS DE RECALL, PRECICSION Y FSCORE
y_pred = svc.predict(X_train_norm)
y_pred_test = svc.predict(X_test_norm)



#Usando el entrenado
print("precision = ", precision_score(y_train, y_pred))
print("recall = ", recall_score(y_train, y_pred))
print("auc = ", roc_auc_score(y_train, y_pred))
print("f1 score = ", f1_score(y_train, y_pred))


confusion_matrix(y_train, y_pred)

KeyboardInterrupt: 

In [ ]:
#Usando el test

confusion_matrix(y_test, y_pred_test)


print("precision = ", precision_score(y_test, y_pred_test))
print("recall = ", recall_score(y_test, y_pred_test))
print("auc = ", roc_auc_score(y_test, y_pred_test))
print("f1 score = ", f1_score(y_test, y_pred_test))